In [5]:
import numpy as np
import pandas as pd

In [6]:
class KNN:
    def __init__(self, k=3, distance_metric='euclidean'):
        self.k = k
        self.distance_metric = distance_metric

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def compute_distance(self, X1, X2):
        if self.distance_metric == 'euclidean':
            return np.linalg.norm(X2 - X1, axis=1)
        elif self.distance_metric == 'manhattan':
            return np.sum(np.abs(X2 - X1), axis=1)

        else:
          raise ValueError("cant find distance metric")

    def predict(self, X):
        return np.apply_along_axis(self.pred, 1, X.values)

    def pred(self, x):
        distances = self.compute_distance(x, self.X_train.values)
        k_indices = np.argpartition(distances, self.k)[:self.k]
        k_nearest_labels = self.y_train.iloc[k_indices]
        return np.mean(k_nearest_labels)


In [7]:
def preprocess_data(train_path, test_path):
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)

    combined_data = pd.concat([train_data, test_data], axis=0, ignore_index=True)

    combined_data = pd.get_dummies(combined_data, columns=['Geography', 'Gender'], drop_first=True)

    features = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary'] + \
               [col for col in combined_data.columns if col.startswith('Geography_') or col.startswith('Gender_')]

    combined_data[features] = (combined_data[features] - combined_data[features].mean()) / combined_data[features].std()

    X_train = combined_data.loc[:len(train_data)-1, features]
    y_train = train_data['Exited']
    X_test = combined_data.loc[len(train_data):, features]

    return X_train, y_train, X_test



In [8]:
def cross_validate(X, y, knn, n_splits=5):
    np.random.seed(42)
    indices = np.random.permutation(len(X))
    fold_size = len(X) // n_splits
    scores = []

    for i in range(n_splits):
        val_indices = indices[i * fold_size:(i + 1) * fold_size]
        train_indices = np.setdiff1d(indices, val_indices)

        X_train, X_val = X.iloc[train_indices], X.iloc[val_indices]
        y_train, y_val = y.iloc[train_indices], y.iloc[val_indices]

        knn.fit(X_train, y_train)
        y_est_prob = knn.predict(X_val)
        score = auc_score(y_val, y_est_prob)
        scores.append(score)

    return scores

def auc_score(y_true, y_est_prob):
    positive_indices = np.where(y_true == 1)[0]
    negative_indices = np.where(y_true == 0)[0]

    if len(positive_indices) == 0 or len(negative_indices) == 0:
        return 0.5

    positive_predictions = y_est_prob[positive_indices]
    negative_predictions = y_est_prob[negative_indices]

    correct_order = np.sum(positive_predictions[:, None] > negative_predictions)
    return correct_order / (len(positive_indices) * len(negative_indices))


In [9]:
# Load and preprocess data
X, y, X_test = preprocess_data('train.csv', 'test.csv')

# Create and evaluate model
knn = KNN(k=5, distance_metric='euclidean')

# Perform cross-validation
cv_scores = cross_validate(X, y, knn)

print("Cross-validation scores:", cv_scores)
print("Mean ROC AUC score:", np.mean(cv_scores))

# Hyperparameter tuning
k_values = [3, 5, 7, 9,11, 13, 15]
distance_metrics = ['euclidean', 'manhattan']
best_score = 0
best_params = {}

for k in k_values:
    for metric in distance_metrics:
        knn = KNN(k=k, distance_metric=metric)
        scores = cross_validate(X, y, knn)
        mean_score = np.mean(scores)

        if mean_score > best_score:
            best_score = mean_score
            best_params = {'k': k, 'distance_metric': metric}

print("Best parameters:", best_params)
print("Best ROC AUC score:", best_score)

# Train on full dataset with optimal hyperparameters and make predictions on test set
knn = KNN(**best_params)
knn.fit(X, y)
test_predictions = knn.predict(X_test)
rounded_predictions = np.round(test_predictions, 2)
pd.DataFrame({'id': pd.read_csv('test.csv')['id'], 'Exited': rounded_predictions.ravel()}).to_csv('submissions.csv', index=False)


Cross-validation scores: [0.7827393099966703, 0.819753998373543, 0.8114446802742773, 0.7861315496098105, 0.8229182427660009]
Mean ROC AUC score: 0.8045975562040603
Best parameters: {'k': 15, 'distance_metric': 'manhattan'}
Best ROC AUC score: 0.8762819122676909
